# Practise exercise 4

## Background


In the wake of the SARS-CoV-2 pandemic, causing COVID-19 disease, international air travel effectively stopped. With cases within many countries declining, understanding how to safely re-open international travel and to which countries is of key importance for the United Kingdom. In particular, it is imperative to determine a risk-based entry strategy, and for that strategy to be flexible in order to quickly react to changing disease prevalence.  Prevalence is defined as the total number of cases existing in a country’s population divided by the total country population worldwide.  

UK government researchers were tasked with estimating the weekly number of infectious travellers from 25 countries that have most flights to the UK. They modelled a country’s prevalence by taking the number of cases in the past 7 days, dividing it by an asymptomatic factor (0.5) and then dividing the result by the total population of each country.  

To implement their model, they turned to software developers to gather the relevant data, extract the key metrics per country and calculate the disease prevalence per week. This Notebook uses four sources of real data:

1. EUnonEUcombinedTravel.css : Travel Data provided by Dr Taylor, but freely available from Eurostat website. In the file there is a row for every airport-country combination with how many passengers are expected in 1 month from that country to that airport. It does not include the country's name, but only their ISO3 codes.

2. ISOcodes.csv: provided by Dr Taylor, this file has the ISO2, ISO3 and country names in it. It is necessary to  match this with the Travel Data to find out which countries the ISO3’s are referring to. 

3. WorldPopData.csv : This is a database of all the world population per country obtained from https://worldpopulationreview.com/countries

4. WHOCOVID.csv : Database from the World Health Organization with information on the most recent COVID numbers per country, obtained from https://covid19.who.int/table


This work is based on the paper: Taylor, R., McCarthy, C. A., Patel, V., Moir, R., Kelly, L., & Snary, E. (2020). The risk of introducing SARS-CoV-2 to the UK via international travel in August 2020. medRxiv.

## Functions used throughout the code

In [1]:
#Function to extract info on CSV. Needs a csv file as input.
def extract_csv(filename):
    # importing csv module 
    import csv 
    # arrays to store data
    fields = [] #for top header, usually strings
    rows = [] #for actual data, mix of strings and numbers

    #read csv file
    # open simplifies exception handling and automatically closes the CSV file.
    with open(filename, "r") as csvfile:
        #create a csv reader object
        # maps the data into lists
        csvreader=csv.reader(csvfile)

        #extract field names through first row
        fields = next(csvreader)

        #extracting each data row one by one
        for row in csvreader:
            rows.append(row)
    
    d= dict();
    d["fields"]=fields
    d["data"]=rows
    return d #returns a dictionary with fields and data

In [2]:
#make dictionary to investigate which are the top 25 travelling countries
#input is travel_list[1]
def add_to_countries(rows):
    max_dictionary={}
    for row in rows:
        if str(row[2]) in max_dictionary.keys():
               max_dictionary[str(row[2])] = max_dictionary[str(row[2])] + int(row[4])
        else:
               max_dictionary.setdefault(str(row[2]), int(row[4]))
        # Used Lambda function, a function without a name, as a sorting mechanism 
        # allows us to sort our dictionary by value
        # reverse=True means that we want our data to be sorted in descending order
    return sorted(max_dictionary.items(), key = lambda x:x[1], reverse=True)

In [3]:
#make dictionary to investigate which are the top 25 UK airports
#input is travel_list[1]

def add_to_airports(rows):
    max_airports={}
    for row in rows:
        if str(row[1]) in max_airports.keys():
            max_airports[str(row[1])] = max_airports[str(row[1])] + int(row[4])
        else: 
            max_airports.setdefault(str(row[1]), int(row[4]))
            
        # Used Lambda function, a function without a name, as a sorting mechanism 
        # allows us to sort our dictionary by value
        # reverse = True means that we want our data to be sorted in descending order
    return sorted(max_airports.items(), key = lambda x:x[1], reverse = True)


In [4]:
#matches the top countries list to the country name
def matchData(list1,list2) :
    newList = []
    i = 0
    j = 0
    while i<len(list1) :
        while j<len(list2[1]) :
            if list1[i][0] == list2[1][j][2]:
                b=str(list1[i][0]),str((list2[1][j][0])),str(list1[i][1])
                newList.append(b)
            j= j+1
        i=i+1
        j=0
    return newList

In [5]:
#needs as input matchedCountries, who_covid_list, world_pop_list
def calculatePrevalence(list1, list2, list3) :

    newList = []
    i = 0
    j = 0
    k = 0
    while i<len(list1) :
        while j<len(list2[1]) :
            while k <len(list3[1]):                       
                if list1[i][1] == list2[1][j][0]:
                    if list2[1][j][0] == list3[1][k][1]:
                        #number of cases in last 7 days/ 0.5*total population*1000 because the CSV misinterprets data
                        prevalence = float(list2[1][j][4])/(0.5*float(list3[1][k][2])*1000)
                        b=str(list1[i][0]),str((list1[i][1])),str(prevalence)
                        newList.append(b)
                k= k + 1
            j= j + 1
            k=0
        i=i+1
        j=0        
    return newList

In [6]:
#sorting function for prevalence
def sortList(list1):
    def getKey(item):
        return item[2]
    return sorted(list1, key = getKey, reverse = True)

In [7]:
#needs as input the travel list and sorted Prevalence
def airportPrevalence (list1, list2):

    top_airports = []
    for i in list1[1] :
        for j in list2:
            #if country code matches with airport
            if j[0] == i[2]:
                b = float(i[4])*float(j[2])
                c = i[1], i[2], b
                top_airports.append(c)
    return top_airports

In [8]:
#Aggregate to get the total number of infected people arriving to each airport 
#input is airport_list

def airports_infected(rows):
    max_airports={ }
    for row in rows:
        if str(row[0]) in max_airports.keys():
            max_airports[str(row[0])] = max_airports[str(row[0])] + int(row[2])
        else: 
            max_airports.setdefault(str(row[0]), int(row[2]))
            
        # Used Lambda function, a function without a name, as a sorting mechanism 
        # allows us to sort our dictionary by value
        # reverse = True means that we want our data to be sorted in descending order
    return sorted(max_airports.items(), key = lambda x:x[1], reverse = True)

In [9]:
#number of infected people arriving from each country 
#input is airport_list

def countries_infected(rows):
    max_airports={ }
    for row in rows:
        if str(row[1]) in max_airports.keys():
            max_airports[str(row[1])] = max_airports[str(row[1])] + int(row[2])
        else: 
            max_airports.setdefault(str(row[1]), int(row[2]))
            
        # Used Lambda function, a function without a name, as a sorting mechanism 
        # allows us to sort our dictionary by value
        # reverse = True means that we want our data to be sorted in descending order
    return sorted(max_airports.items(), key = lambda x:x[1], reverse = True)

## Research Questions

1. Which are the top 25 countries with COVID travelling by air into the UK? 

2. Which are the top 25 UK airports people use to travel? 

3. What is the disease prevalence to the top 25 countries travelling into the UK? 

4. What is the number of infected people from the top 25 countries arriving at each UK airport? 

5. a. What is the total number of infected people arriving to each airport? b. What is the total number of infected people arriving from each country in the top 25? 

In [10]:
#1. Investigate which are the top 25 countries travelling into the UK
#make sure they include country code, country names and passengers 

# Get CSV data
# csv file name 
travelData = "EUnonEUcombinedTravel.csv" #provided by Dr Taylor
countryCode ="ISOcodes.csv"  #provided by Dr Taylor
worldPopData = "WorldPopData.csv" #obtained from https://worldpopulationreview.com/countries
whoCOVID = "WHOCOVID.csv" # obtained from https://covid19.who.int/table

#TIP: mac and pc have different ways of accessing a file

#Get dictionaries out of the data (keys:fields, values:data)
travel_dict = extract_csv(travelData)
world_pop_dict = extract_csv(worldPopData)
who_covid_dict = extract_csv(whoCOVID)
country_code_dict = extract_csv(countryCode)

#get lists of the data in dictionaries
travel_list = list(travel_dict.values())
world_pop_list = list(world_pop_dict.values())
who_covid_list = list(who_covid_dict.values())
country_code_list = list(country_code_dict.values())

#manipulate data
sorted_countries= add_to_countries(travel_list[1])
top_countries = sorted_countries [:25]
matchedCountries= matchData(top_countries,country_code_list)
matchedCountries

[('ESP', 'Spain', '2706321'),
 ('USA', 'United States of America', '1124137'),
 ('ITA', 'Italy', '864412'),
 ('FRA', 'France', '746692'),
 ('GRC', 'Greece', '723248'),
 ('DEU', 'Germany', '651094'),
 ('IRL', 'Ireland', '607160'),
 ('PRT', 'Portugal', '531283'),
 ('NLD', 'Netherlands', '468559'),
 ('TUR', 'Turkey', '422079'),
 ('POL', 'Poland', '383807'),
 ('ARE', 'United Arab Emirates', '375013'),
 ('CAN', 'Canada', '233581'),
 ('CHE', 'Switzerland', '223895'),
 ('CYP', 'Cyprus', '207526'),
 ('HRV', 'Croatia', '175919'),
 ('ROU', 'Romania', '172180'),
 ('DNK', 'Denmark', '163288'),
 ('BGR', 'Bulgaria', '140800'),
 ('IND', 'India', '130743'),
 ('HUN', 'Hungary', '124460'),
 ('SWE', 'Sweden', '124270'),
 ('QAT', 'Qatar', '119105'),
 ('NOR', 'Norway', '113565'),
 ('CZE', 'Czech Republic', '104123')]

In [11]:
#2. Investigate which are the top 25 UK airports people use to travel 
sorted_airports = add_to_airports(travel_list[1])
top_airports = sorted_airports [:25]
top_airports

[('LONDON HEATHROW airport', 3648223),
 ('LONDON GATWICK airport', 2337904),
 ('MANCHESTER airport', 1516365),
 ('LONDON STANSTED airport', 1372224),
 ('LONDON LUTON airport', 892184),
 ('BIRMINGHAM airport', 641923),
 ('EDINBURGH airport', 494420),
 ('BRISTOL airport', 438491),
 ('GLASGOW airport', 285216),
 ('EAST MIDLANDS airport', 283101),
 ('NEWCASTLE airport', 247822),
 ('LEEDS BRADFORD airport', 238488),
 ('LIVERPOOL airport', 218280),
 ('LONDON/CITY airport', 184703),
 ('BELFAST/ALDERGROVE airport', 156932),
 ('SOUTHEND airport', 111425),
 ('CARDIFF airport', 93162),
 ('DONCASTER SHEFFIELD airport', 81437),
 ('ABERDEEN/DYCE airport', 48726),
 ('BOURNEMOUTH airport', 47806),
 ('PRESTWICK airport', 41029),
 ('SOUTHAMPTON airport', 37805),
 ('EXETER airport', 35727),
 ('NORWICH airport', 20162),
 ('BELFAST/CITY airport', 11399)]

In [12]:
#3. Calculate the disease prevalence to the top 25 countries travelling into the UK. 
# Use sorted countries to get population, number of cases. prevalence = cases/(0.5*population)
# prevalence should be a number between 0 and 1.

prevalenceCountries = calculatePrevalence(matchedCountries, who_covid_list, world_pop_list)
sortedPrevalence = sortList(prevalenceCountries)
sortedPrevalence

[('IRL', 'Ireland', '0.01007860607972885'),
 ('SWE', 'Sweden', '0.008164950617693467'),
 ('PRT', 'Portugal', '0.007174275543216934'),
 ('NLD', 'Netherlands', '0.006574545756746826'),
 ('CYP', 'Cyprus', '0.006448786152254631'),
 ('CHE', 'Switzerland', '0.005390183418755897'),
 ('DNK', 'Denmark', '0.004948722437511675'),
 ('HRV', 'Croatia', '0.0038194836048422672'),
 ('ITA', 'Italy', '0.003775340824142493'),
 ('POL', 'Poland', '0.0033476709446983244'),
 ('FRA', 'France', '0.0032218582511977945'),
 ('DEU', 'Germany', '0.0028994100086625193'),
 ('CAN', 'Canada', '0.00277461641431488'),
 ('ROU', 'Romania', '0.002622144206391505'),
 ('HUN', 'Hungary', '0.0025309639370246486'),
 ('ARE', 'United Arab Emirates', '0.0024933263582208286'),
 ('TUR', 'Turkey', '0.0021703109426145298'),
 ('BGR', 'Bulgaria', '0.001697933854265235'),
 ('NOR', 'Norway', '0.0013491375867628833'),
 ('QAT', 'Qatar', '0.0009885274585368614'),
 ('GRC', 'Greece', '0.0008590572398454426'),
 ('IND', 'India', '0.000188521140097

In [13]:
#4.	Calculate the number of infected people arriving into each airport from top 25 countries
#by multiplying number of passengers from each country by prevalence in that country

airport_list = airportPrevalence(travel_list,sortedPrevalence)
infected_airports = sortList(airport_list)
infected_airports

[('LONDON HEATHROW airport', 'IRL', 1168.322095368248),
 ('LONDON GATWICK airport', 'ITA', 894.6425150970465),
 ('LONDON STANSTED airport', 'IRL', 791.8156080478175),
 ('LONDON HEATHROW airport', 'DEU', 754.1104485630433),
 ('LONDON GATWICK airport', 'IRL', 739.9107867372138),
 ('LONDON GATWICK airport', 'PRT', 719.0848119721766),
 ('LONDON STANSTED airport', 'ITA', 707.1326623843613),
 ('MANCHESTER airport', 'IRL', 549.818197467448),
 ('LONDON HEATHROW airport', 'CHE', 520.8588139378011),
 ('LONDON GATWICK airport', 'FRA', 520.1819020888887),
 ('LONDON STANSTED airport', 'PRT', 503.5193547251373),
 ('BIRMINGHAM airport', 'IRL', 492.17864929747867),
 ('LONDON HEATHROW airport', 'ITA', 479.0794245612099),
 ('MANCHESTER airport', 'PRT', 475.5468543821345),
 ('LONDON HEATHROW airport', 'FRA', 472.78192349726675),
 ('LONDON HEATHROW airport', 'SWE', 451.2849855905356),
 ('LONDON HEATHROW airport', 'NLD', 444.0250967734104),
 ('LONDON HEATHROW airport', 'ARE', 443.5901857174258),
 ('LONDON 

In [14]:
#5.a.Aggregate to get the total number of infected people arriving from the 
# top 25 countries arriving at each UK airport

infected_arriving = airports_infected(infected_airports)
infected_arriving

[('LONDON HEATHROW airport', 6333),
 ('LONDON GATWICK airport', 5095),
 ('LONDON STANSTED airport', 3982),
 ('MANCHESTER airport', 3307),
 ('LONDON LUTON airport', 2446),
 ('BIRMINGHAM airport', 1640),
 ('EDINBURGH airport', 1496),
 ('BRISTOL airport', 1190),
 ('LONDON/CITY airport', 790),
 ('GLASGOW airport', 758),
 ('LIVERPOOL airport', 690),
 ('EAST MIDLANDS airport', 538),
 ('LEEDS BRADFORD airport', 534),
 ('NEWCASTLE airport', 532),
 ('SOUTHEND airport', 332),
 ('BELFAST/ALDERGROVE airport', 277),
 ('CARDIFF airport', 207),
 ('DONCASTER SHEFFIELD airport', 193),
 ('ABERDEEN/DYCE airport', 173),
 ('SOUTHAMPTON airport', 164),
 ('EXETER airport', 84),
 ('BOURNEMOUTH airport', 75),
 ('INVERNESS airport', 60),
 ('NORWICH airport', 59),
 ('BELFAST/CITY airport', 47),
 ('PRESTWICK airport', 44),
 ('HUMBERSIDE airport', 39),
 ('NEWQUAY airport', 36),
 ('DURHAM TEES VALLEY airport', 27),
 ('SUMBURGH airport', 0),
 ('KIRKWALL airport', 0),
 ('DUNDEE airport', 0),
 ('STORNOWAY airport', 0)

In [15]:
#5.b. Calculate the number of infected people arriving from each top 25 countries 

country_arriving = countries_infected(airport_list)
matchedArrivals= matchData(country_arriving, country_code_list)
matchedArrivals

[('IRL', 'Ireland', '6107'),
 ('PRT', 'Portugal', '3802'),
 ('ITA', 'Italy', '3253'),
 ('NLD', 'Netherlands', '3068'),
 ('FRA', 'France', '2395'),
 ('DEU', 'Germany', '1879'),
 ('CYP', 'Cyprus', '1328'),
 ('POL', 'Poland', '1274'),
 ('CHE', 'Switzerland', '1201'),
 ('SWE', 'Sweden', '1009'),
 ('ARE', 'United Arab Emirates', '931'),
 ('TUR', 'Turkey', '908'),
 ('DNK', 'Denmark', '800'),
 ('HRV', 'Croatia', '663'),
 ('CAN', 'Canada', '646'),
 ('GRC', 'Greece', '609'),
 ('ROU', 'Romania', '447'),
 ('HUN', 'Hungary', '311'),
 ('BGR', 'Bulgaria', '231'),
 ('NOR', 'Norway', '148'),
 ('QAT', 'Qatar', '114'),
 ('IND', 'India', '24'),
 ('ESP', 'Spain', '0')]